## download original database 

In [1]:
! git clone https://github.com/MassBank/MassBank-data.git

fatal: destination path 'MassBank-data' already exists and is not an empty directory.


In [2]:
import os
from glob import glob
import sys
from multiprocessing import Pool
import numpy as np
import pickle



from tqdm import tqdm

In [2]:
vender_list = glob("./MassBank-data/*")
file_list = []
for ven in tqdm(vender_list):
    file_list.extend([os.path.abspath(one) for one in glob(ven+"/*.txt")])

print("number of file list",len(file_list))

100%|██████████| 45/45 [00:00<00:00, 66.91it/s]

number of file list 56270


In [3]:
NUMBET_OF_MP_PROCESSES = 3 # Number of processes use to parse and expand data.

AUTHORS = "AUTHORS"
AC_INSTRUMENT = "AC$INSTRUMENT"
AC_INSTRUMENT_TYPE = "AC$INSTRUMENT_TYPE"
CH_NAME = "CH$NAME"
CH_SMILES = "CH$SMILES"
AC_MASS_SPECTROETRY = "AC$MASS_SPECTROMETRY"
MS_TYPE = "MS_TYPE"
ION_MODE = "ION_MODE"
IONIZATION_ENERGY = "IONIZATION_ENERGY"

def analyze(args):
    num,path = args
    #param init 
    auth = None
    inst = None
    inst_type = None
    name = None
    smiles = None
    ms_type = None
    ion_mode = None
    ret = {}
    with open(path) as f:
        temp = f.read().split("\n")
        for num,line in enumerate(temp):
            row = line.split(": ",1)
            if row[0] == AUTHORS:
                ret["authors"]= row[1]
            elif row[0] == AC_INSTRUMENT:
                ret["instrument"] = row[1]
            elif row[0] == AC_INSTRUMENT_TYPE:
                ret["instrument_type"] = row[1]
            elif row[0] == CH_NAME:
                ret["name"] = row[1]
            elif row[0] == CH_SMILES:
                ret["smiles"] = row[1]
            elif row[0] == AC_MASS_SPECTROETRY:
                label,_type = row[1].split(" ",1)
                if label == MS_TYPE:
                    ret["ms_type"] = _type
                elif label == ION_MODE:
                    ret["ion_mode"] = _type
                elif label == IONIZATION_ENERGY:
                    ret["ionization_energy"] = _type
            elif row[0] == "PK$PEAK":
                peak_start = num+1
        peak_x,peak_y = zip(*[one.split(" ")[2:4] for one in temp[peak_start:-1] if one != "//"])
        ret["peak_x"] = np.array(peak_x,dtype=np.float32)
        ret["peak_y"] = np.array(peak_y,dtype=np.float32)
    ret["path"] = path
    return ret

with Pool(processes=NUMBET_OF_MP_PROCESSES) as p:
        ret = p.map(analyze,enumerate(file_list))

print("number of expand data:",len(ret))

import pickle
with open("massbank.pkl","wb") as f:
    pickle.dump(ret,f,protocol=2)

number of expand data: 56270


In [3]:
with open("massbank.pkl","rb") as f:
    li = pickle.load(f)

In [5]:
def is_select(one):
    return one["smiles"]!="N/A" and \
        one["ms_type"]=="MS" and \
        one["instrument_type"]=="EI-B" and \
        "ionization_energy" in one and \
        one["ionization_energy"]=="70 eV" and \
        one["ion_mode"]=="POSITIVE"

def is_select_new(one):
    return one["smiles"]!="N/A" and \
        one["ms_type"]=="MS" and \
        one["instrument_type"]=="EI-B" and \
        "ionization_energy" in one and \
        one["ionization_energy"]=="70 eV" and \
        one["ion_mode"]=="POSITIVE" and \
        "2H" not in one["smiles"]

ret = [one for one in li if is_select(one)]
print(len(ret))
ret = [one for one in li if is_select_new(one)]
print(len(ret))

7463
7316
